In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

POSTGRES_DATABASE = os.getenv('POSTGRES_DATABASE')
POSTGRES_HOST = os.getenv('POSTGRES_HOST')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
POSTGRES_URL = os.getenv('POSTGRES_URL')
POSTGRES_URL_NON_POOLING = os.getenv('POSTGRES_URL_NON_POOLING')
POSTGRES_URL_NO_SSL = os.getenv('POSTGRES_URL_NO_SSL')
POSTGRES_USER = os.getenv('POSTGRES_USER')

In [2]:
# pegar os dados de /models_results
import pandas as pd

dfPortfolioNumbers = pd.read_csv('models_results/portfolio_numbers.csv')

dfPortfolio = pd.read_csv('models_results/portfolio.csv')

dfModelResults = pd.read_csv('models_results/model_results.csv')

yahoo_data = pd.read_csv('yahoo_data/yahoo_data.csv')

fundamentus_data = pd.read_csv('fundamentus_data/fundamentus.csv')

back_test_results = pd.read_csv('models_results/backtest_results.csv')

In [3]:
fundamentus_data['stock'] = fundamentus_data['papel']
dfPortfolio = pd.merge(dfPortfolio, fundamentus_data[['stock', 'descricao_papel']], how='left', on='stock')

dfModelResults['stock'] = dfModelResults['Ticker']
dfModelResults['Model'] = dfModelResults['Algorithm']
dfPortfolio = pd.merge(dfPortfolio, dfModelResults[['stock', 'Model', 'MAPE']], how='left', on=['stock', 'Model'])

In [4]:
stocksTikers = list(dfPortfolio['stock'].unique())
# para todos os tikers de stocksTikers, pegar os dados de yahoo_data para 'stock', 'date' e 'Close' e colocar em um novo DataFrame
dfHistoricalData = pd.DataFrame()
for stock in stocksTikers:
    dfHistoricalData = pd.concat([dfHistoricalData, yahoo_data[yahoo_data['stock'] == stock][['stock','date', 'Close']]], axis=0)
# filtrar para datas de até 30 dias atrás da maior data
biggestDate = dfHistoricalData['date'].max()
dfHistoricalData = dfHistoricalData[dfHistoricalData['date'] > str(pd.Timestamp(biggestDate) - pd.Timedelta(days=30))]

In [5]:
dfPortfolio.head()

,date,stock,Prediction,Model,Margin_prediction,Sharpe,Sortino,descricao_papel,MAPE
0,1,HYPE3,25.993537,Random Forest,0.254168,-0.078244,-0.113063,HYPERA,0.258988
1,2,HYPE3,25.798980,Random Forest,0.254168,-0.078244,-0.113063,HYPERA,0.258988
2,3,HYPE3,25.510808,Random Forest,0.254168,-0.078244,-0.113063,HYPERA,0.258988
3,4,HYPE3,25.510808,Random Forest,0.254168,-0.078244,-0.113063,HYPERA,0.258988
4,5,HYPE3,25.309105,Random Forest,0.254168,-0.078244,-0.113063,HYPERA,0.258988


In [6]:
dfHistoricalData.head()

,stock,date,Close
21222,HYPE3,2024-10-24,27.740000
21223,HYPE3,2024-10-25,26.200001
21224,HYPE3,2024-10-28,23.920000
21225,HYPE3,2024-10-29,24.090000
21226,HYPE3,2024-10-30,24.100000


In [7]:
import psycopg2

conn = psycopg2.connect(
    host=POSTGRES_HOST,
    database=POSTGRES_DATABASE,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD
)

apagarStockAnalisys = "DELETE FROM \"StocksAnalisys\";"
apagarStockPredictions = "DELETE FROM \"StocksPredictions\";"
apagarStocksHistory = "DELETE FROM \"StocksHistory\";"
apagarModelHistory = "DELETE FROM \"ModelHistory\";"

cur = conn.cursor()
cur.execute(apagarStockAnalisys)
cur.execute(apagarStockPredictions)
cur.execute(apagarStocksHistory)
cur.execute(apagarModelHistory)
conn.commit()

In [8]:
# Inserir os dados historicos
for index, row in dfHistoricalData.iterrows():
    sql = "INSERT INTO \"StocksHistory\" (\"date\", \"code\", \"price\") VALUES (%s, %s, %s);"
    val = (row['date'], row['stock'], row['Close'])
    cur.execute(sql, val)
conn.commit()

In [9]:
dates = []
for i in range(5):
    dates.append(str(pd.Timestamp(biggestDate) + pd.Timedelta(days=i + 3))[:10])

In [10]:
# inserir as previsões
for index, row in dfPortfolio.iterrows():
    sql = "INSERT INTO \"StocksPredictions\" (\"date\", \"code\", \"name\", \"price\") VALUES (%s, %s, %s, %s);"
    val = (dates[row['date']-1], row['stock'], row['descricao_papel'], row['Prediction'])
    cur.execute(sql, val)
conn.commit()

In [11]:
for stock in stocksTikers:
    line = dfPortfolio[dfPortfolio['stock'] == stock].iloc[0]
    sql = "INSERT INTO \"StocksAnalisys\" (\"code\", \"name\", \"index1\", \"index2\", \"index3\", \"index4\", \"index5\", \"index6\") VALUES (%s, %s, %s, %s, %s, %s, %s, %s);"
    val = (stock, line['descricao_papel'], line['Prediction'], line['Margin_prediction'], line['Sharpe'], line['Sortino'], line['MAPE'], 100//line['Prediction'])
    cur.execute(sql, val)
conn.commit()

In [12]:
portfolioMargin = dfPortfolioNumbers['Value'].iloc[0]
portfolioSharpe = dfPortfolioNumbers['Value'].iloc[1]
portfolioSortino = dfPortfolioNumbers['Value'].iloc[2]
sql = "INSERT INTO \"Portfolio\" (\"date\", \"index1\", \"index2\", \"index3\") VALUES (%s, %s, %s, %s);"
val = (biggestDate, portfolioMargin, portfolioSharpe, portfolioSortino)
cur.execute(sql, val)
conn.commit()

In [13]:
results = list(back_test_results[back_test_results['Backtest Results'] != 0]['Backtest Results'])
for i in range(len(results)):
    date = str(pd.Timestamp(biggestDate) - pd.Timedelta(days=i + 7))[:10]
    sql = "INSERT INTO \"ModelHistory\" (\"date\", \"index1\") VALUES (%s, %s);"
    val = (date, results[len(results) - i - 1])
    cur.execute(sql, val)
conn.commit()